检查每个claim，第一个检索结果如果是 factcheck.afp.com 或者 reuters.com ，
然后再核对title是否对应的上claim：
找claim第一个单词，再title里找到对应的起始位置，然后终止位置为...
查看title是否在claim中出现，如果是，那么FALSE改为TRUE


In [8]:
import json
import sys
dirs = ["../..", "../create_prompt_llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
import prompt_rag

search_engine = "brave"

def load_train_bak():
    with open("covmis_data/train.json.bak", "r") as f:
        return json.load(f)
    
def load_train():
    with open("covmis_data/train.json", "r") as f:
        return json.load(f)
    
def load_type1():
    with open("output/type_1.json", "r") as f:
        return json.load(f)

def load_type2():
    with open("output/type_2.json", "r") as f:
        return json.load(f)
    
def save_type1(x):
    with open("output/type_1.json", "w") as f:
        json.dump(x, f, indent=4)
        
def save_type2(x):
    with open("output/type_2.json", "w") as f:
        json.dump(x, f, indent=4)

def item_exist(x, data):
    for item in data:
        if item["id"] == x["id"]:
            return True
    return False

def update_data_type1(x, data: list):

    data_type2 = load_type2()
    if item_exist(x, data_type2):
        return
    
    if not item_exist(x, data):
        data.append(x)

def update_data_type2(x, data: list):

    if not item_exist(x, data):
        data.append(x)

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)
try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    pass
    # data_search_llm = [{
    #     "claim": i["claim"],
    #     "claimant": i["claimant"],
    #     "label": i["label"],
    #     "date": i["date"],
    # } for i in data_search]
    # with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
    #     json.dump(data_search_llm, f, indent=4)

def save_diff(x, model_name, diff_version):
    with open(f"output/diff_{model_name}_v{diff_version}.json", "w") as f:
        json.dump(x, f, indent=4)

def load_diff(model_name, diff_version):
    try:
        with open(f"output/diff_{model_name}_v{diff_version}.json", "r") as f:
            return json.load(f)
    except:
        data_init = [{"claim": i["claim"], "label": int(i["label"])} for i in data_search_llm]
        with open(f"output/diff_{model_name}_v{diff_version}.json", "w") as f:
            json.dump(data_init, f, indent=4)
        return data_init


In [2]:
cnt = 0
for item in data_search:
    claim = item["claim"]
    title = item[f"{search_engine}_search_results"]["web"]["results"][0]["title"]
    url = item[f"{search_engine}_search_results"]["web"]["results"][0]["url"]
    check_list = ["poynter"]

    ok = False
    for site in check_list:
        if site in url:
            ok = True
            break
    if not ok:
        continue
    cnt += 1

    print(claim)
    break
cnt


Dr Mikovits, an established scientist, revealed that Dr Anthony Fauci was paid to cover up the extent of the problem caused by COVID-19. Also she revealed that SARS-CoV-2 was created in a lab and that influenza vaccines increase COVID-19 mortality by 36%.


1

In [7]:
s = "Information Summary:\n\nInformation 1, published on 2020-02-11, provides guidelines on what to do if you have been around someone with COVID-19. It includes information on symptoms, testing, and daily activities.\n\nInformation 2, published on 2020-05-27, offers guidance for close contacts of people with COVID-19. It advises daily LFD tests for 7 days if you have been in close contact with an infected person. It also provides information on when to end isolation and how to protect immunocompromised individuals.\n\nInformation 3, with no publication date, explains how to get tested for COVID-19, when to test, and what to expect.\n\nInformation 4, with no publication date, provides details on what to do if you are a close contact to someone with COVID-19, including seeking medical care and considering contacting a healthcare provider.\n\nInformation 5, with no publication date, defines close contacts based on exposure risks and advises contacting a healthcare provider for treatment options.\n\nRestated CLAIM: As of March 19, 2020, it is possible to contract the coronavirus without having direct contact with an infected person.\n\nReasoning and Evidence:\n\nThe claim is actually incorrect based on the available information. The earliest publication date is 2020-02-11, which is before the claim's publication date. None of the provided information supports the claim that it is possible to contract the coronavirus without having direct contact with an infected person. In fact, Information 2, published on 2020-05-27, specifically states that if you have had close contact with someone who has tested positive for COVID-19, you are at higher risk of becoming infected yourself. Therefore, the claim is FALSE based on the available information and current scientific understanding of COVID-19 transmission."




print(s)


Information Summary:

Information 1, published on 2020-02-11, provides guidelines on what to do if you have been around someone with COVID-19. It includes information on symptoms, testing, and daily activities.

Information 2, published on 2020-05-27, offers guidance for close contacts of people with COVID-19. It advises daily LFD tests for 7 days if you have been in close contact with an infected person. It also provides information on when to end isolation and how to protect immunocompromised individuals.

Information 3, with no publication date, explains how to get tested for COVID-19, when to test, and what to expect.

Information 4, with no publication date, provides details on what to do if you are a close contact to someone with COVID-19, including seeking medical care and considering contacting a healthcare provider.

Information 5, with no publication date, defines close contacts based on exposure risks and advises contacting a healthcare provider for treatment options.

Resta

添加related url

In [15]:
# data_train = load_train()

# for item in data_train:
#     related_url = ""
#     article_id = item["related_articles"][-1]
#     with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/articles/{article_id}.txt", "r") as f:
#         lines = f.readlines()
    
#     s = ""
#     now = len(lines) - 1
#     while now >=0 and lines[now].strip == "":
#         now -= 1

#     s = lines[now]
#     st = s.find("article_url:")
#     if st != -1:
#         related_url = s[st + len("article_url:"):].strip()
#     item["related_url"] = related_url


# # with open("covmis_data/train.json", "w") as f:
# #     json.dump(data_train, f, indent=4)



In [6]:
diff_data = load_diff("llama3", 1)
data_train = load_train()
data_type1 = load_type1()
data_type2 = load_type2()

claim_dict = {}
for item in diff_data:
    if item["match2"].lower() == "no":
        claim_dict[item["claim"].strip()] = 1

cnt = 0
cnt2 = 0
for i, item in enumerate(data_search):
    claim = item["claim"]
    if int(data_train[i]["label"]) == 1:
        continue
    # ok = False
    for search_result_id in range(2):
        if len(item[f"{search_engine}_search_results"]["web"]["results"]) == search_result_id:
            break
        title_bak = item[f"{search_engine}_search_results"]["web"]["results"][search_result_id]["title"]
        title = title_bak
        url = item[f"{search_engine}_search_results"]["web"]["results"][search_result_id]["url"]
        check_list = [
            "factcheck.afp.com", "reuters.com", 
            "africacheck.org"
        ]

        ok = False
        for site in check_list:
            if site in url:
                ok = True
                break
        if not ok:
            continue

        # cnt += 1
        pos_st = title.find(claim.split(' ')[0])
        if pos_st == -1:
            continue
        pos_en = len(title)
        if title.endswith(" ..."):
            pos_en -= 4
        elif title.endswith("..."):
            pos_en -= 3 

        title = title[pos_st : pos_en]
        pos_en = len(title)

        # ["| Reuters", "| Fact Check", "| Fact", "|"]
        pos_verical_bar = title.rfind("|")
        if pos_verical_bar != -1:
            pos_en = pos_verical_bar
        # elif "africacheck.org" in url:
        #     pos_verical_bar = title.rfind("-")
        #     if pos_verical_bar != -1:
        #         pos_en = pos_verical_bar

        if len(title[:pos_en].split()) >= 5 and title[:pos_en].strip() in claim:
            if claim_dict.get(claim.strip()) is not None:
                cnt += 1
                # print(claim)
                if data_train[i]["label"] == 0 and not item_exist(data_train[i], data_type2):
                    data_train[i]["label"] = 2
                    update_data_type1(data_train[i], data_type1)
                    # print(claim)
                else:
                    pass
            else:
                cnt2 += 1
                
                if data_train[i]["label"] == 0:
                    
                    if not title_bak.lower().startswith("false claim"):
                        if (title_bak.lower().startswith("fact check:") and "reuters.com" in url) or \
                            ("africacheck.org" in url):
                            data_train[i]["label"] = 2
                            update_data_type1(data_train[i], data_type1)
                            # print(claim)
                        else:
                            pass
                else:
                    pass
            break
# Covid-19 not caused by bacteria, aspirin not a cure - Africa Check
cnt, cnt2, cnt + cnt2


(609, 154, 763)

In [14]:
data_type1 = load_type1()
data_type2 = load_type2()


def get_error_labels(data_type):
    error_cnt = 0
    tot_cnt = 0
    data_bak = load_train_bak()
    for item in data_type:
        if item["label"] == 1:
            continue
        for i in data_bak:
            if i["id"] == item["id"] and i["label"] != item["label"]:
                error_cnt += 1
                break
        tot_cnt += 1
        
    return error_cnt, tot_cnt
print(len(data_type2))
get_error_labels(data_type1)[1] + get_error_labels(data_type2)[1], \
    get_error_labels(data_type1)[0] + get_error_labels(data_type2)[0]


408


(1285, 931)

In [13]:
# (12192 - 931) / 12192

99 / 92.5 * 100


0.9236384514435696

In [4]:
# save_type1(data_type1)

# with open("covmis_data/train.json", "w") as f:
#     json.dump(data_train, f, indent=4)

# len(data_type1)


703